In [1]:
val LOC = "/home/tbrownex/data/Hackett/archive/Mastercard/"
val FILE = "Egypt.csv"

LOC = /home/tbrownex/data/Hackett/archive/Mastercard/
FILE = Egypt.csv


Egypt.csv

In [2]:
val df = spark.read.format("csv")
         .option("header", "false")   // Keep the headers: "true" skips them
         .option("mode", "DROPMALFORMED")
         .load(LOC+FILE)

df = [_c0: string, _c1: string ... 4 more fields]


[_c0: string, _c1: string ... 4 more fields]

In [3]:
df.describe().show()

+-------+-----+--------------------+------------+--------------------+----------+--------------------+
|summary|  _c0|                 _c1|         _c2|                 _c3|       _c4|                 _c5|
+-------+-----+--------------------+------------+--------------------+----------+--------------------+
|  count|45901|               45901|       45901|               45901|     45901|               45901|
|   mean| null|                null|        null|                null|      null|   5308318.084986665|
| stddev| null|                null|        null|                null|      null|  2.21777945948177E7|
|    min|Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-01-01|-0.00000000069849...|
|    max|Egypt|Small Business Pr...|GG0000008024|Processed-Issuing...|2018-05-01|        9999.9780935|
+-------+-----+--------------------+------------+--------------------+----------+--------------------+



In [4]:
val formatter = java.text.NumberFormat.getIntegerInstance
print("Data has " +  formatter.format(df.count())+" rows and " + formatter.format(df.columns.length)+" columns:\n")

// print Column names
val colList = df.columns

for( a <- colList){
 println( " " + a )
 }

Data has 45,901 rows and 6 columns:
 _c0
 _c1
 _c2
 _c3
 _c4
 _c5


formatter = java.text.DecimalFormat@674dc
colList = Array(_c0, _c1, _c2, _c3, _c4, _c5)


Array(_c0, _c1, _c2, _c3, _c4, _c5)

In [5]:
df.show(5)      // equivalent to "head"

+-----+--------------------+------------+--------------------+----------+----------------+
|  _c0|                 _c1|         _c2|                 _c3|       _c4|             _c5|
+-----+--------------------+------------+--------------------+----------+----------------+
|Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-01-01|284533.954567226|
|Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-02-01|263907.051428571|
|Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-03-01|351187.960412742|
|Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-04-01|308846.532766667|
|Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-05-01| 258098.75879821|
+-----+--------------------+------------+--------------------+----------+----------------+
only showing top 5 rows



In [10]:
print(df.first())      // Get the first row

[Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-01-01,284533.954567226]

In [11]:
// Iterate over some rows
val first4 = df.take(4)
for (x <- first4) {
    println(x)
}

[Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-01-01,284533.954567226]
[Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-02-01,263907.051428571]
[Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-03-01,351187.960412742]
[Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-04-01,308846.532766667]


first4 = Array([Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-01-01,284533.954567226], [Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-02-01,263907.051428571], [Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-03-01,351187.960412742], [Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-04-01,308846.532766667])


Array([Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-01-01,284533.954567226], [Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-02-01,263907.051428571], [Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-03-01,351187.960412742], [Egypt,All Cirrus Issuing Programs,GG0000000044,Processed-Acquiring Domestic Cash Volume,2015-04-01,308846.532766667])

In [6]:
// rename ALL columns
val newNames = Seq("country", "product", "account","type", "month", "amount")
val df2 = df.toDF(newNames: _*)
df2.printSchema

root
 |-- country: string (nullable = true)
 |-- product: string (nullable = true)
 |-- account: string (nullable = true)
 |-- type: string (nullable = true)
 |-- month: string (nullable = true)
 |-- amount: string (nullable = true)



newNames = List(country, product, account, type, month, amount)
df2 = [country: string, product: string ... 4 more fields]


[country: string, product: string ... 4 more fields]

In [7]:
// rename a SINGLE column
val df3 = df2.withColumnRenamed("account", "customer")
df3.printSchema

root
 |-- country: string (nullable = true)
 |-- product: string (nullable = true)
 |-- customer: string (nullable = true)
 |-- type: string (nullable = true)
 |-- month: string (nullable = true)
 |-- amount: string (nullable = true)



df3 = [country: string, product: string ... 4 more fields]


[country: string, product: string ... 4 more fields]

In [9]:
// Add a column
import org.apache.spark.sql.functions.lit    // needed if you're adding the same value for each row
val newDF = df3.withColumn("new", lit(10))    // adding value 10 to all records

newDF = [country: string, product: string ... 5 more fields]


[country: string, product: string ... 5 more fields]

In [16]:
newDF.show(5)

+-------+--------------------+------------+--------------------+----------+----------------+---+
|country|             product|    customer|                type|     month|          amount|new|
+-------+--------------------+------------+--------------------+----------+----------------+---+
|  Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-01-01|284533.954567226| 10|
|  Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-02-01|263907.051428571| 10|
|  Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-03-01|351187.960412742| 10|
|  Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-04-01|308846.532766667| 10|
|  Egypt|All Cirrus Issuin...|GG0000000044|Processed-Acquiri...|2015-05-01| 258098.75879821| 10|
+-------+--------------------+------------+--------------------+----------+----------------+---+
only showing top 5 rows



In [10]:
// Filter a numeric
val before = newDF.count()
val filtered = newDF.filter($"amount" > 500000.0)      // Filter on a column
val after = filtered.count()

println("Removed " + formatter.format( before-after) + " rows. " + formatter.format(after) +" remaining")

Removed 23,877 rows. 22,024 remaining


before = 45901
filtered = [country: string, product: string ... 5 more fields]
after = 22024


22024

In [11]:
// Filter an alpha
val before = newDF.count()
val filtered = newDF.filter($"country" === "Egypt")
val after = filtered.count()

println("Removed " + formatter.format( before-after) + " rows. " + formatter.format(after) +" remaining")

Removed 0 rows. 45,901 remaining


before = 45901
filtered = [country: string, product: string ... 5 more fields]
after = 45901


45901

#### Nulls

In [154]:
// check if a column has Nulls
df2.filter("amount is null").show

+-------+-------+-------+----+-----+------+
|country|product|account|type|month|amount|
+-------+-------+-------+----+-----+------+
+-------+-------+-------+----+-----+------+



In [13]:
// count the number of nulls
println(df2.filter($"amount".isNull).count)

0


In [1]:
// create data with Null
case class MyRow(x: Double, y:String)
val data = Seq(MyRow(1.1, "bat"),MyRow(2.2, "cat"),MyRow(Double.NaN, "dog"))

defined class MyRow
data = List(MyRow(1.1,bat), MyRow(2.2,cat), MyRow(NaN,dog))


List(MyRow(1.1,bat), MyRow(2.2,cat), MyRow(NaN,dog))

In [2]:
val df = data.toDF("id", "typ")
df.show

+---+---+
| id|typ|
+---+---+
|1.1|bat|
|2.2|cat|
|NaN|dog|
+---+---+



df = [id: double, typ: string]


[id: double, typ: string]

In [178]:
// replaces Null with a static value
val t = df.na.fill(18)
t.show

+----+---+
|  id|typ|
+----+---+
| 1.1|bat|
| 2.2|cat|
|18.0|dog|
+----+---+



t = [id: double, typ: string]


[id: double, typ: string]

In [179]:
// equivalent to "dropna"
val t = df.na.drop()
t.show

+---+---+
| id|typ|
+---+---+
|1.1|bat|
|2.2|cat|
+---+---+



t = [id: double, typ: string]


[id: double, typ: string]

In [5]:
val col = Array("id")

col = Array(id)


Array(id)

In [6]:
import org.apache.spark.ml.feature.Imputer

val imputer = new Imputer()
  .setInputCols(col)
  .setOutputCols(col.map(c => s"${c}_imputed")
  .setStrategy("mean")

Name: Syntax Error.
Message: 
StackTrace: 

In [183]:
imputer.fit(df).transform(df)

Name: java.lang.IllegalArgumentException
Message: requirement failed: Column typ must be of type equal to one of the following types: [double, float] but was actually of type string.
StackTrace:   at scala.Predef$.require(Predef.scala:224)
  at org.apache.spark.ml.util.SchemaUtils$.checkColumnTypes(SchemaUtils.scala:60)
  at org.apache.spark.ml.feature.ImputerParams$$anonfun$2.apply(Imputer.scala:76)
  at org.apache.spark.ml.feature.ImputerParams$$anonfun$2.apply(Imputer.scala:74)
  at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
  at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
  at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
  at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
  at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
  at scala.collection.mutable.ArrayOps$ofRef.map(ArrayOps.scala:186)
  at org.apache.spark.ml.feature.Impute

#### GroupBy

In [81]:
val LOC = "/home/tbrownex/data/"
val FILE = "spark.csv"

val df = spark.read.format("csv")
         .option("header", "true")   // whether to keep the headers
         .option("inferSchema", "true")
         .option("mode", "DROPMALFORMED")
         .load(LOC+FILE)

df.show

+---+--------+------+
| ID|Customer|Amount|
+---+--------+------+
|  1|       A|   100|
|  2|       B|   200|
|  3|       A|   300|
|  4|       B|   400|
|  1|       A|   100|
+---+--------+------+



LOC = /home/tbrownex/data/
FILE = spark.csv
df = [ID: int, Customer: string ... 1 more field]


[ID: int, Customer: string ... 1 more field]

In [83]:
// get the number of rows per group
df.groupBy("Customer").count().show

+--------+-----+
|Customer|count|
+--------+-----+
|       B|    2|
|       A|    3|
+--------+-----+



In [151]:
// now descending
val g = df.groupBy("Customer")
g.count().orderBy($"count".desc).show

+--------+-----+
|Customer|count|
+--------+-----+
|       A|    3|
|       B|    2|
+--------+-----+



g = RelationalGroupedDataset: [grouping expressions: [Customer: string], value: [ID: int, Customer: string ... 1 more field], type: GroupBy]


RelationalGroupedDataset: [grouping expressions: [Customer: string], value: [ID: int, Customer: string ... 1 more field], type: GroupBy]

In [153]:
// which customer has the highest Avg?
val metric = g.avg("Amount").orderBy("avg(Amount)")

println("Customer"+ " Avg")
metric.collect.foreach( (x =>  println(x(0)+" "+formatter.format(x(1)))))

Customer Avg
A 167
B 300


metric = [Customer: string, avg(Amount): double]


[Customer: string, avg(Amount): double]

In [ ]:
// get a column
val col = df.select("word").collect()

In [19]:
df.reduce {
    (x, y) =>
    if (x.getAs[Int]("timestamp") > y.getAs[Int]("timestamp")) x else y
}

23676

In [ ]:
df.orderBy($"amount".desc).show(1)